In [1]:
import torch
import pdb
from torch import nn
import torch.nn.functional as f
from torchsummary import summary

In [58]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels,out_channels,strides=2,downsample=False):
        super(ResNetBlock, self).__init__()
        self.downsample = downsample
        self.conv1 = nn.Conv2d(in_channels,out_channels,stride=strides,kernel_size=3,padding=1,bias=False)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,out_channels,stride=1,kernel_size=3,padding=1,bias=False)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(in_channels,out_channels,stride=strides,kernel_size=1,padding=0,bias=False)
        
    def forward(self,x):
        residual = x
        x = f.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        if self.downsample == True:
            residual = self.bn2(self.conv3(residual))
        x = x + residual
        x = f.relu(x)
        return x

In [59]:
# check resnet block
net = ResNetBlock(32,64,downsample=True)

In [60]:
net.to('cuda')
summary(net,input_size=(32,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          18,432
       BatchNorm2d-2         [-1, 64, 112, 112]             128
            Conv2d-3         [-1, 64, 112, 112]          36,864
       BatchNorm2d-4         [-1, 64, 112, 112]             128
            Conv2d-5         [-1, 64, 112, 112]           2,048
       BatchNorm2d-6         [-1, 64, 112, 112]             128
Total params: 57,728
Trainable params: 57,728
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 6.12
Forward/backward pass size (MB): 36.75
Params size (MB): 0.22
Estimated Total Size (MB): 43.10
----------------------------------------------------------------


In [114]:
class ResNet(nn.Module):
    def __init__(self,in_channels,base_channels,output,block):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels,base_channels,stride=2,kernel_size=7,padding=3,bias=False)
        self.bn1   = nn.BatchNorm2d(base_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layer1  = self._make_layer(block,3,False,base_channels,base_channels)
        self.layer2  = self._make_layer(block,4,True,base_channels,base_channels*2)
        self.layer3  = self._make_layer(block,6,True,base_channels*2,base_channels*4)
        self.layer4  = self._make_layer(block,3,True,base_channels*4,base_channels*8)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dense   = nn.Linear(base_channels*8,output)
    
    def _make_layer(self,block,no_layer,downsample,in_channels,out_channels):
        layers = []
        if downsample == True:
            layers.append(block(in_channels,out_channels,downsample=True))
        else:
            layers.append(block(in_channels,out_channels,strides=1))
            
        for _ in range(no_layer-1):
            layers.append(block(out_channels,out_channels,strides=1))
            
        return nn.Sequential(*layers)
    
    def forward(self,x):
        x = f.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = f.relu(self.dense(x))
        return x

In [117]:
net = ResNet(3,64,1000,ResNetBlock)

In [118]:
net.to('cuda')
summary(net,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]          36,864
       BatchNorm2d-5           [-1, 64, 56, 56]             128
            Conv2d-6           [-1, 64, 56, 56]          36,864
       BatchNorm2d-7           [-1, 64, 56, 56]             128
       ResNetBlock-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,864
      BatchNorm2d-10           [-1, 64, 56, 56]             128
           Conv2d-11           [-1, 64, 56, 56]          36,864
      BatchNorm2d-12           [-1, 64, 56, 56]             128
      ResNetBlock-13           [-1, 64, 56, 56]               0
           Conv2d-14           [-1, 64,